## notebook only

In [1]:
# kedro
from kedro.framework.context import load_context
proj_path = '../../../' 
context = load_context(proj_path)
# df = catalog.load("XXX")
parameters = context.params

# import from src
sys.path.append("./../../../src/forecast_keiba/")
from data.scraping_netkeiba import get_race_url_by_year_and_mon
from data.scraping_netkeiba import get_race_url
from data.scraping_netkeiba import scrape_race_span
from data.scraping_netkeiba import scrape_id
from data.scraping_netkeiba import scrape_race_info
from data.scraping_netkeiba import scrape_race_results
from data.scraping_netkeiba import scrape_horse_results
from data.scraping_netkeiba import HorseResults
from data.scraping_netkeiba import scrape_peds
from data.scraping_netkeiba import add_blood_data
from data.scraping_netkeiba import scrape_jockey_results
from data.scraping_netkeiba import get_race_data

## import文
メイン関数実行のため

In [2]:
import datetime
import pytz
import re
import time
import os
from os import path
from selenium import webdriver
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from pandas.plotting import scatter_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler

## メイン関数

In [3]:
def scraping_netkeiba(parameters):
    print("start scraping_netkeiba")
    # race urlの全量を取得
    get_race_url(parameters)
    
    url_dir = '../../../data/raw/race_url/'

    if not os.path.exists(url_dir):
        os.makedirs(url_dir)
    
    dt_now = datetime.datetime.now()
    race_url_list = []
    for year in range(parameters["scrape_year_start"], parameters["scrape_year_end"]+1):
        if year != dt_now.year:
            for month in range(1,13):
                path = url_dir+str(year)+'-'+str(month)+'.txt'
                with open(path) as f:
                    race_url_list += f.readlines()
        else:
            for month in range(1,dt_now.month):
                path = url_dir+str(year)+'-'+str(month)+'.txt'
                with open(path) as f:
                    race_url_list += f.readlines()
    race_id_list = []
    for url in race_url_list:
        race_id_list.append(url[-14:-2])
    race_id_list = race_id_list[:parameters["scraping_limit"]]
        
    # 初回判定
    if os.path.exists('../../../data/raw/race_results_df.pickle') != True:
        flag = False
        add_blood = get_race_data(race_id_list,flag)
        print("FINISH!!!")
        return

    race_results_df = pd.read_pickle('../../../data/raw/race_results_df.pickle')
    got_race_id_list = set(list(race_results_df.index))
    difference_id_list = set(race_id_list) ^ got_race_id_list
    
    #なぜか失敗する
    if '201305030305' in difference_id_list:
        difference_id_list.remove('201305030305')
    if '201805010107' in difference_id_list:
        difference_id_list.remove('201805010107')
    if '201709050706' in difference_id_list:
        difference_id_list.remove('201709050706')
    if '201808030406' in difference_id_list:
        difference_id_list.remove('201808030406')
    if '201005050810' in difference_id_list:
        difference_id_list.remove('201005050810')
    if '201009040210' in difference_id_list:
        difference_id_list.remove('201009040210')
    if '201006040811' in difference_id_list:
        difference_id_list.remove('201006040811')
    if '201008060611' in difference_id_list:
        difference_id_list.remove('201008060611')

        
    if len(difference_id_list) > 0:
        flag = True
        race_results_dif_df = get_race_data(difference_id_list,flag)
        race_results_df = pd.concat([race_results_df, race_results_dif_df])
        race_results_df.to_pickle('../../../data/raw/race_results_df.pickle')
        return race_results_df
    else:
        flag = False
        return

In [4]:
def main(parameters):
    return scraping_netkeiba(parameters)

if __name__ == "__main__":
    main(parameters)

start scraping_netkeiba


/opt/forecast-keiba/src/forecast_keiba/pipelines/base/../../data/scraping_netkeiba.py:117: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver', chrome_options=options) # 去年までのデータ


get_race_url_by_year_and_mon year: 2020 month: 9
already have 120 urls (2020 9)
レース結果取得中
scrape_race_results race_id: 201905010201
scrape_race_results race_id: 201905010202
scrape_race_results race_id: 201905010203
馬情報取得中
ジョッキー情報取得中


/opt/conda/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


データ結合中
血統情報取得中
FINISH!!!
